In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import joblib

In [2]:
historical_path = r"E:\OIL_InfySpringboard\Code\oil and gas.csv"
df_historical = pd.read_csv(historical_path, parse_dates=["Date"])

competitor_path = r"E:\OIL_InfySpringboard\Code\competitor-dataset.csv"
df_competitor = pd.read_csv(competitor_path, parse_dates=["Date"])

scraped_prices_path = r"E:\OIL_InfySpringboard\Code\scraped_oil_prices.csv"
df_scraped = pd.read_csv(scraped_prices_path, parse_dates=["Date"])

In [3]:
df = pd.merge(df_historical, df_competitor, on=["Date", "Symbol"], how="left")
df = pd.merge(df, df_scraped, on=["Date", "Symbol"], how="left")


In [4]:
df

,Symbol,Date,Open,High,Low,Close,Volume,Currency,Company,Closing Price_x,Selling Price,Volume Sold_x,Closing Price_y,Volume Sold_y
0,Brent Oil,2000-01-04,23.9000,24.7000,23.8900,24.3900,32509,USD,ExxonMobil,24.3900,28.570195,8161,NaN,NaN
1,Brent Oil,2000-01-04,23.9000,24.7000,23.8900,24.3900,32509,USD,Chevron,24.3900,29.057995,8024,NaN,NaN
2,Brent Oil,2000-01-04,23.9000,24.7000,23.8900,24.3900,32509,USD,ConocoPhillips,24.3900,28.814095,8092,NaN,NaN
3,Brent Oil,2000-01-04,23.9000,24.7000,23.8900,24.3900,32509,USD,Marathon Petroleum,24.3900,28.326295,8231,NaN,NaN
4,Brent Oil,2000-01-05,24.2500,24.3700,23.7000,23.7300,30310,USD,ExxonMobil,23.7300,27.822476,7609,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92091,Heating Oil,2022-06-16,4.5320,4.5825,4.4124,4.5713,28269,USD,Marathon Petroleum,4.5713,7.477914,7002,NaN,NaN
92092,Heating Oil,2022-06-17,4.5719,4.6444,4.2936,4.3398,0,USD,ExxonMobil,4.3398,7.089614,0,NaN,NaN
92093,Heating Oil,2022-06-17,4.5719,4.6444,4.2936,4.3398,0,USD,Chevron,4.3398,7.046216,0,NaN,NaN
92094,Heating Oil,2022-06-17,4.5719,4.6444,4.2936,4.3398,0,USD,ConocoPhillips,4.3398,7.133012,0,NaN,NaN


In [5]:
df.rename(columns={"Closing Price_x": "Closing Price"}, inplace=True)
df.drop(columns=["Closing Price_y", "Close", "Currency", "Open", "High", "Low"], inplace=True, errors="ignore")

In [6]:
df

,Symbol,Date,Volume,Company,Closing Price,Selling Price,Volume Sold_x,Volume Sold_y
0,Brent Oil,2000-01-04,32509,ExxonMobil,24.3900,28.570195,8161,NaN
1,Brent Oil,2000-01-04,32509,Chevron,24.3900,29.057995,8024,NaN
2,Brent Oil,2000-01-04,32509,ConocoPhillips,24.3900,28.814095,8092,NaN
3,Brent Oil,2000-01-04,32509,Marathon Petroleum,24.3900,28.326295,8231,NaN
4,Brent Oil,2000-01-05,30310,ExxonMobil,23.7300,27.822476,7609,NaN
...,...,...,...,...,...,...,...,...
92091,Heating Oil,2022-06-16,28269,Marathon Petroleum,4.5713,7.477914,7002,NaN
92092,Heating Oil,2022-06-17,0,ExxonMobil,4.3398,7.089614,0,NaN
92093,Heating Oil,2022-06-17,0,Chevron,4.3398,7.046216,0,NaN
92094,Heating Oil,2022-06-17,0,ConocoPhillips,4.3398,7.133012,0,NaN


In [7]:
df = df[["Date", "Symbol", "Closing Price", "Selling Price"]]

In [8]:
df.columns

Index(['Date', 'Symbol', 'Closing Price', 'Selling Price'], dtype='object')

In [9]:
df

,Date,Symbol,Closing Price,Selling Price
0,2000-01-04,Brent Oil,24.3900,28.570195
1,2000-01-04,Brent Oil,24.3900,29.057995
2,2000-01-04,Brent Oil,24.3900,28.814095
3,2000-01-04,Brent Oil,24.3900,28.326295
4,2000-01-05,Brent Oil,23.7300,27.822476
...,...,...,...,...
92091,2022-06-16,Heating Oil,4.5713,7.477914
92092,2022-06-17,Heating Oil,4.3398,7.089614
92093,2022-06-17,Heating Oil,4.3398,7.046216
92094,2022-06-17,Heating Oil,4.3398,7.133012


In [10]:
df.fillna(method="ffill", inplace=True)
df.fillna(0, inplace=True)

# 🔹 Convert Date to UNIX Timestamp
df["Date"] = df["Date"].astype("int64") // 10**9

C:\Users\Admin\AppData\Local\Temp\ipykernel_9316\2591228479.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_9316\2591228479.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna(method="ffill", inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_9316\2591228479.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna(0, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_9316\2591228479.py:5: SettingWithCopyWarning: 


In [11]:
oil_types = df["Symbol"].unique().tolist()


In [12]:
oil_types

['Brent Oil', 'Crude Oil WTI', 'Natural Gas', 'Heating Oil']

In [13]:
ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
ohe.fit(np.array(oil_types).reshape(-1, 1))
symbol_encoded = ohe.transform(df[["Symbol"]])
encoded_cols = [f"OilType_{oil.replace(' ', '_')}" for oil in ohe.categories_[0]]
df_encoded = pd.DataFrame(symbol_encoded, columns=encoded_cols)

c:\Users\Admin\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(


In [14]:
df_encoded

,OilType_Brent_Oil,OilType_Crude_Oil_WTI,OilType_Heating_Oil,OilType_Natural_Gas
0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0
...,...,...,...,...
92091,0.0,0.0,1.0,0.0
92092,0.0,0.0,1.0,0.0
92093,0.0,0.0,1.0,0.0
92094,0.0,0.0,1.0,0.0


In [15]:
df = pd.concat([df, df_encoded], axis=1).drop(columns=["Symbol"])

In [16]:
df

,Date,Closing Price,Selling Price,OilType_Brent_Oil,OilType_Crude_Oil_WTI,OilType_Heating_Oil,OilType_Natural_Gas
0,946944000,24.3900,28.570195,1.0,0.0,0.0,0.0
1,946944000,24.3900,29.057995,1.0,0.0,0.0,0.0
2,946944000,24.3900,28.814095,1.0,0.0,0.0,0.0
3,946944000,24.3900,28.326295,1.0,0.0,0.0,0.0
4,947030400,23.7300,27.822476,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
92091,1655337600,4.5713,7.477914,0.0,0.0,1.0,0.0
92092,1655424000,4.3398,7.089614,0.0,0.0,1.0,0.0
92093,1655424000,4.3398,7.046216,0.0,0.0,1.0,0.0
92094,1655424000,4.3398,7.133012,0.0,0.0,1.0,0.0


In [17]:
features = ["Date", "Closing Price"] + encoded_cols
target = "Selling Price"
X = df[features]
y = df[target]

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 🔹 Scale Features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [19]:
xgb_model = XGBRegressor(n_estimators=100, random_state=42)
xgb_model.fit(X_train_scaled, y_train)

# XGBoost Predictions
xgb_train_preds = xgb_model.predict(X_train_scaled).reshape(-1, 1)
xgb_test_preds = xgb_model.predict(X_test_scaled).reshape(-1, 1)

In [20]:
scaler_preds = StandardScaler()
xgb_train_preds_scaled = scaler_preds.fit_transform(xgb_train_preds)
xgb_test_preds_scaled = scaler_preds.transform(xgb_test_preds)

In [21]:
X_train_lstm = np.reshape(xgb_train_preds_scaled, (xgb_train_preds_scaled.shape[0], 1, 1))
X_test_lstm = np.reshape(xgb_test_preds_scaled, (xgb_test_preds_scaled.shape[0], 1, 1))

In [22]:
lstm_model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(1, 1)),
    LSTM(50),
    Dense(1)
])
lstm_model.compile(optimizer='adam', loss='mse')

c:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [23]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

lstm_model.fit(X_train_lstm, y_train, epochs=100, batch_size=16, validation_data=(X_test_lstm, y_test), callbacks=[early_stopping])

Epoch 1/100
4605/4605 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - loss: 1173.6676 - val_loss: 37.6922
Epoch 2/100
4605/4605 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - loss: 16.8593 - val_loss: 1.6963
Epoch 3/100
4605/4605 ━━━━━━━━━━━━━━━━━━━━ 16s 4ms/step - loss: 1.0511 - val_loss: 0.5667
Epoch 4/100
4605/4605 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - loss: 0.5337 - val_loss: 0.5551
Epoch 5/100
4605/4605 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - loss: 0.5144 - val_loss: 0.4924
Epoch 6/100
4605/4605 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - loss: 0.5230 - val_loss: 0.5942
Epoch 7/100
4605/4605 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - loss: 0.5164 - val_loss: 0.6272
Epoch 8/100
4605/4605 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - loss: 0.5054 - val_loss: 0.5284
Epoch 9/100
4605/4605 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - loss: 0.5187 - val_loss: 0.5827
Epoch 10/100
4605/4605 ━━━━━━━━━━━━━━━━━━━━ 19s 4ms/step - loss: 0.5308 - val_loss: 0.5894
Epoch 11/100
4605/4605 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - loss: 0.5105 - val_loss: 0.6376
Epo

In [24]:
lstm_preds = lstm_model.predict(X_test_lstm)
mae = mean_absolute_error(y_test, lstm_preds)
r2 = r2_score(y_test, lstm_preds)
print(f"✅ LSTM Model - MAE: {mae:.2f}, R² Score: {r2:.2f}")

576/576 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
✅ LSTM Model - MAE: 0.41, R² Score: 1.00


In [25]:
lstm_model.save("lstm_oil_price_model2.h5")
joblib.dump(scaler, "feature_scaler2.pkl")
joblib.dump(scaler_preds, "prediction_scaler2.pkl")
joblib.dump(xgb_model, "xgb_oil_price_model2.pkl")
joblib.dump(ohe, "ohe_encoder2.pkl")
print("✅ Models saved successfully!")

✅ Models saved successfully!


In [26]:
lstm_model.save("lstm_oil_price_model3.h5")

# (Optional) Save only the model weights
lstm_model.save_weights("lstm_oil_price.weights.h5")
